# Object tracking using Bayesian Tracker

This notebook shows how to previously localised and classified objects from a series of mask images and unite them over time using Bayesian multi-object tracking (btrack). 

The sections of this notebook are as follows:

1. Load segmentation objects and filter based on size
2. Run btrack, uniting the objects locations over time
3. Batch process

The data used in this notebook is timelapse microscopy data with h2b-gfp/rfp markers that show the spatial extent of the nucleus and it's mitotic state. 

In [1]:
import btrack
from tqdm import tqdm
import os
import glob

## 1. Load segmentation objects

In [2]:
expt = 'ND0025'
pos = 'Pos6'
root_dir = '/home/nathan/data/kraken/ras'

In [3]:
with btrack.dataio.HDF5FileHandler(
     f'{root_dir}/{expt}/{pos}/objects.h5', 'r', obj_type='obj_type_1',
) as hdf:
    objects = hdf.objects

Check objects feature classification

In [4]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob,area,class id,eccentricity,mean_intensity,prob_anaphase,prob_apoptosis,prob_interphase,prob_metaphase,prob_prometaphase
0,0,1384.249268,1.825763,0.0,0,False,0,3,0.0,96.0,1.0,0.960572,27.317749,1.0,2.408825e-11,2.937358e-23,6.532457e-19,1.428292e-31


Excluding segments that are too small to feasibly be cells

In [5]:
objects = [o for o in objects if o.properties['area']>100.]

## 2. Run btrack  

Unite each object with it's subsequent position at the following time point and export as a tracking file

In [8]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(
        './models/MDCK_config_wildtype.json'
    )
    tracker.max_search_radius = 40

    # append the objects to be tracked
    tracker.append(objects)

    # set the volume
    tracker.volume=((0, 1200), (0, 1600), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(f'{root_dir}/{expt}/{pos}/tracks.h5', obj_type='obj_type_1')

    # get the tracks in a format for napari visualization (optional)
    visaulise_tracks, properties, graph = tracker.to_napari(ndim=2)
    
    tracks = tracker.tracks

[INFO][2022/05/18 06:15:54 PM] btrack (v0.4.3) library imported
[INFO][2022/05/18 06:15:54 PM] Setting max XYZ search radius to: 100
[INFO][2022/05/18 06:15:54 PM] Starting BayesianTracker session
[INFO][2022/05/18 06:15:54 PM] Loading motion model: MDCK_motion
[INFO][2022/05/18 06:15:54 PM] Setting max XYZ search radius to: 40
[INFO][2022/05/18 06:15:55 PM] Set volume to ((0, 1200), (0, 1600), (-100000.0, 100000.0))
[INFO][2022/05/18 06:15:55 PM] Starting tracking... 
[INFO][2022/05/18 06:15:55 PM] Tracking objects in frames 0 to 99 (of 1183)...
[INFO][2022/05/18 06:15:55 PM] Tracking objects in frames 100 to 199 (of 1183)...
[INFO][2022/05/18 06:15:56 PM] Tracking objects in frames 200 to 299 (of 1183)...
[INFO][2022/05/18 06:15:56 PM] Tracking objects in frames 300 to 399 (of 1183)...
[INFO][2022/05/18 06:15:57 PM] Tracking objects in frames 400 to 499 (of 1183)...
[INFO][2022/05/18 06:15:58 PM] Tracking objects in frames 500 to 599 (of 1183)...
[INFO][2022/05/18 06:16:00 PM] Tracki

In [10]:
[track.fate.name for track in tracks]

['TERMINATE_LAZY',
 'TERMINATE_LAZY',
 'DIVIDE',
 'TERMINATE_LAZY',
 'TERMINATE_LAZY',
 'TERMINATE_BORDER',
 'TERMINATE_LAZY',
 'TERMINATE_BORDER',
 'TERMINATE_LAZY',
 'DIVIDE',
 'DIVIDE',
 'DIVIDE',
 'DIVIDE',
 'DIVIDE',
 'DIVIDE',
 'TERMINATE_BACK',
 'TERMINATE_BORDER',
 'TERMINATE_BORDER',
 'DIVIDE',
 'TERMINATE_BORDER',
 'TERMINATE_BORDER',
 'TERMINATE_LAZY',
 'TERMINATE_BORDER',
 'DIVIDE',
 'DIVIDE',
 'TERMINATE_BORDER',
 'TERMINATE_BORDER',
 'TERMINATE_BORDER',
 'DIVIDE',
 'TERMINATE_BACK',
 'TERMINATE_BORDER',
 'DIVIDE',
 'TERMINATE_BORDER',
 'TERMINATE_BORDER',
 'TERMINATE_LAZY',
 'DIVIDE',
 'TERMINATE_BACK',
 'TERMINATE_BORDER',
 'DIVIDE',
 'DIVIDE',
 'TERMINATE_LAZY',
 'TERMINATE_BORDER',
 'TERMINATE_BORDER',
 'DIVIDE',
 'DIVIDE',
 'TERMINATE_BORDER',
 'DIVIDE',
 'APOPTOSIS',
 'TERMINATE_LAZY',
 'TERMINATE_LAZY',
 'APOPTOSIS',
 'TERMINATE_BORDER',
 'FALSE_POSITIVE',
 'TERMINATE_BORDER',
 'TERMINATE_LAZY',
 'TERMINATE_BORDER',
 'TERMINATE_BORDER',
 'DIVIDE',
 'DIVIDE',
 'TERMI

In [8]:
btrack.__version__

'0.4.3'

In [9]:
stop

NameError: name 'stop' is not defined

# 3. Batch process
Iterate over many experiments and positions (need to ensure you define normalisation and classification functions above first)

In [ ]:
root_dir = '/home/nathan/data'
expt_list = ['ND0009', 'ND0010', 'ND0011']
pos_list = 'all'
overwrite = False

for expt in expt_list:
    
        # Find all positions in that experiment, if pos_list is all then it finds all positions
        if pos_list == 'all':
            pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                    if 'Pos' in pos 
                    and os.path.isdir(f'{root_dir}/{expt}/{pos}')]  
            
        ### Iterate over all positions in that experiment
        for pos in pos_list:
            
            ### check if overwrite param is false check if raw directory already created and if type of transform file already exists and decide whether to skip pos
            if not overwrite and glob.glob(f'{root_dir}/{expt}/{pos}/*tracks*.h5'):
                print(glob.glob(f'{root_dir}/{expt}/{pos}/*tracks*.h5'), f'file found, skipping {expt}/{pos}')
                continue
                
            print(f'Starting {expt}/{pos}')
            
            # load objects
            with btrack.dataio.HDF5FileHandler(
                 f'{root_dir}/{expt}/{pos}/objects.h5', 'r', obj_type='obj_type_1',
            ) as hdf:
                objects = hdf.objects
            
            # filter objects for non-cell type sizes
            objects = [o for o in objects if o.properties['area']>100.]

            # initialise a tracker session using a context manager
            with btrack.BayesianTracker() as tracker:

                # configure the tracker using a config file
                tracker.configure_from_file(
                    './models/MDCK_config_wildtype.json'
                )
                tracker.max_search_radius = 40

                # append the objects to be tracked
                tracker.append(objects)

                # set the volume
                tracker.volume=((0, 1200), (0, 1600), (-1e5, 1e5))

                # track them (in interactive mode)
                tracker.track_interactive(step_size=100)

                # generate hypotheses and run the global optimizer
                tracker.optimize()

                tracker.export(f'{root_dir}/{expt}/{pos}/tracks.h5', obj_type='obj_type_1')
                
            print(f'Finished {expt}/{pos}')